This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_uploads/pytorch-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: Pytorch Multiple Input Output

The following tutorial demonstrates how to upload a Pytorch Multiple Input Output model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a Pytorch Multiple Input Output to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'pytorch-multi-io'
pipeline_name = f'pytorch-multi-io'

model_name = 'pytorch-multi-io'
model_file_name = "./models/model-auto-conversion_pytorch_multi_io_model.pt"

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for PyTorch models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a PyTorch model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, PyTorch model Required*) | Set as the `Framework.PyTorch`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The input schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The output schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
| `convert_wait` | `bool` (*Upload Method Optional, PyTorch model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input_1', pa.list_(pa.float32(), list_size=10)),
    pa.field('input_2', pa.list_(pa.float32(), list_size=5))
])
output_schema = pa.schema([
    pa.field('output_1', pa.list_(pa.float32(), list_size=3)),
    pa.field('output_2', pa.list_(pa.float32(), list_size=2))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.PYTORCH`.

In [7]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,pytorch-multi-io
Version,f2f97177-62f7-4124-8c7a-5e458612310d
File Name,model-auto-conversion_pytorch_multi_io_model.pt
SHA,792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-12-Apr 17:28:11


In [8]:
model.config().runtime()

'onnx'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [9]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [10]:
# clear the pipeline if it was used before
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.0.252',
   'name': 'engine-c844b6f6b-bftpr',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pytorch-multi-io',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'pytorch-multi-io',
      'sha': '792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8',
      'status': 'Running',
      'version': 'f2f97177-62f7-4124-8c7a-5e458612310d'}]}}],
 'engine_lbs': [{'ip': '10.28.0.253',
   'name': 'engine-lb-d7cc8fc9c-2knjz',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [11]:
mock_inference_data = [np.random.rand(10, 10), np.random.rand(10, 5)]
mock_dataframe = pd.DataFrame(
    {
        "input_1": mock_inference_data[0].tolist(),
        "input_2": mock_inference_data[1].tolist(),
    }
)

In [12]:
pipeline.infer(mock_dataframe)

,time,in.input_1,in.input_2,out.output_1,out.output_2,anomaly.count
0,2024-04-12 17:28:32.625,"[0.9109700519, 0.3169575485, 0.0769643673, 0.8...","[0.8953973272, 0.2605879044, 0.7480816126, 0.2...","[0.02766981, -0.07259649, 0.2620843]","[-0.14548574, 0.11293939]",0
1,2024-04-12 17:28:32.625,"[0.8466220739, 0.5009051624, 0.1561670987, 0.7...","[0.9767897887, 0.4272101532, 0.5284931864, 0.7...","[-0.0552716, -0.0068743527, 0.20660618]","[-0.07974212, 0.051745012]",0
2,2024-04-12 17:28:32.625,"[0.6028804613, 0.191810007, 0.4115756608, 0.04...","[0.8267110995, 0.1623875745, 0.7075612713, 0.3...","[0.020370252, -0.013679747, 0.22586967]","[0.18796784, -0.07283032]",0
3,2024-04-12 17:28:32.625,"[0.7153226706, 0.5041547823, 0.3013886164, 0.1...","[0.1366878788, 0.1746995667, 0.8863207683, 0.7...","[-0.013608955, -0.09129709, 0.4607787]","[0.05677295, 0.050741404]",0
4,2024-04-12 17:28:32.625,"[0.3411437764, 0.7328673708, 0.980411164, 0.32...","[0.5961747145, 0.7028441849, 0.4200162769, 0.9...","[-0.059214503, -0.036837004, 0.2780401]","[-0.0002666721, -0.088903986]",0
5,2024-04-12 17:28:32.625,"[0.4542408627, 0.116382546, 0.1009935686, 0.90...","[0.9132025402, 0.7122236805, 0.9662095863, 0.9...","[0.04246279, -0.15800561, 0.4424684]","[-0.046756577, -0.0058508515]",0
6,2024-04-12 17:28:32.625,"[0.8305691738, 0.204106362, 0.7037219991, 0.20...","[0.3193820444, 0.5499977325, 0.3284488511, 0.1...","[-0.047178756, 0.021341052, 0.1612856]","[0.011848757, -0.06673317]",0
7,2024-04-12 17:28:32.625,"[0.472728972, 0.9336368358, 0.3760646219, 0.84...","[0.0560057682, 0.4189214525, 0.9670296607, 0.3...","[-0.047954347, -0.0315362, 0.27049458]","[0.15487051, 0.008990034]",0
8,2024-04-12 17:28:32.625,"[0.6893380018, 0.9131849658, 0.7052689101, 0.3...","[0.0508924326, 0.9863486789, 0.196386472, 0.04...","[-0.100792035, 0.04375423, 0.14693232]","[0.16369253, 0.040995926]",0
9,2024-04-12 17:28:32.625,"[0.9433111673, 0.4592229624, 0.8921336148, 0.3...","[0.5270989968, 0.9967014918, 0.7570415345, 0.6...","[-0.14226617, -0.051928118, 0.25710666]","[-0.004090662, 0.044676527]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [13]:
pipeline.undeploy()

name,pytorch-multi-io
created,2024-04-12 17:21:40.944555+00:00
last_updated,2024-04-12 17:28:18.633501+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"a2f7ca99-2c2f-4dcd-ae18-7d95abe93ae9, d4644148-28ab-486f-aad6-f00581aae424"
steps,pytorch-multi-io
published,False
